<a href="https://colab.research.google.com/github/Cheryl0905/Datathon/blob/main/Olivia_RSM8432_Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# Libraries
from sklearn.cluster import KMeans
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

ImportError: cannot import name 'LogisticClassifier' from 'sklearn.linear_model' (/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/__init__.py)

In [17]:
df = pd.read_excel('/content/SQL_Tushar.xlsx', sheet_name='Sheet1')
y = df['Fraud']
X = df.drop(columns=['Fraud','no_attending_physicians'])

In [23]:
y = y.map({'Yes': 1, 'No': 0})

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
# Scale the data
scaler = MinMaxScaler()
cols_to_exclude = ['PID', 'AttendingPhysician']
cols_to_scale = [col for col in X_train.columns if col not in cols_to_exclude]
scaled_X_train = X_train[cols_to_scale]
scaled_X_train = pd.DataFrame(scaler.fit_transform(scaled_X_train), columns=scaled_X_train.columns)
scaled_X_train

,total_no_of_claims,total_amount_reimbursed,avg_inpatient_time,no_unique_beneficiaries,No_of_E_Cases,No_of_V_Cases,no_infectious,no_neoplasms,no_endocrine,no_blood,...,no_genitourinary.1,no_pregnancy.1,no_skin.1,no_musculoskeletal.1,no_congenital.1,no_perinatal.1,no_symptoms.1,no_injury.1,Fraud.1,Total
0,0.059740,0.045774,0.139286,0.074919,0.0,0.071429,0.0,0.0,0.066667,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.642857
1,0.000000,0.000704,0.085714,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
2,0.000000,0.002113,0.057143,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.005195,0.005164,0.152381,0.006515,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.142857
4,0.000000,0.001643,0.085714,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8330,0.000000,0.003286,0.085714,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
8331,0.000000,0.000704,0.257143,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8332,0.000000,0.001878,0.228571,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
8333,0.000000,0.000939,0.028571,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000


In [26]:
cols_to_scale = [col for col in X_test.columns if col not in cols_to_exclude]
scaled_X_test = X_test[cols_to_scale]
scaled_X_test = pd.DataFrame(scaler.fit_transform(scaled_X_test), columns=scaled_X_test.columns)
scaled_X_test

,total_no_of_claims,total_amount_reimbursed,avg_inpatient_time,no_unique_beneficiaries,No_of_E_Cases,No_of_V_Cases,no_infectious,no_neoplasms,no_endocrine,no_blood,...,no_genitourinary.1,no_pregnancy.1,no_skin.1,no_musculoskeletal.1,no_congenital.1,no_perinatal.1,no_symptoms.1,no_injury.1,Fraud.1,Total
0,0.000000,0.008629,0.114286,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000000
1,0.005155,0.005393,0.057143,0.005435,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.071429
2,0.000000,0.003236,0.085714,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.002697,0.342857,0.000000,0.0,0.0,0.0,0.0,0.0,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000
4,0.000000,0.004854,0.200000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3568,0.067010,0.090497,0.171429,0.070652,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.357143
3569,0.000000,0.006472,0.571429,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
3570,0.005155,0.006472,0.071429,0.005435,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.071429
3571,0.237113,0.232445,0.125836,0.250000,0.0,0.0,0.0,0.0,0.2,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.500000


# Model:

In [29]:
model = LogisticRegression()
model.fit(scaled_X_train, y_train)

LogisticRegression()

In [34]:
predictions = model.predict(scaled_X_test)
predictions

array([1, 0, 0, ..., 1, 1, 1])

In [36]:
accuracy = accuracy_score(y_test,predictions)
accuracy

1.0

In [31]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2303
           1       1.00      1.00      1.00      1270

    accuracy                           1.00      3573
   macro avg       1.00      1.00      1.00      3573
weighted avg       1.00      1.00      1.00      3573



In [32]:
print(confusion_matrix(y_test, predictions))

[[2303    0]
 [   0 1270]]
